# ChatGPT Prompts with Clarifai

![Clarifai logo](https://www.clarifai.com/hs-fs/hubfs/logo/Clarifai/clarifai-740x150.png?width=240)

This is a self-contained notebook that shows how to run the Clarifai Recognition model with your account credentials, and start creating fun prompts for ChatGPT using your images. This notebook has been done manually for simplicity, but all these features can be automated at scale for use in digital asset management applications.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Clarifai/colab-notebooks/blob/main/general/Inference_on_image_to_GPT_prompt.ipynb)

## Get Started 

*  Install Clarifai gRPC client for Python and the latest protobuf

In [1]:
!pip install -q clarifai-grpc && pip install --upgrade --no-deps -q protobuf 

*   Install other dependencies



In [2]:
import os
from io import BytesIO
import skimage 
import IPython.display
import matplotlib.pyplot as plt
import urllib.request
from PIL import Image
import numpy as np

%matplotlib inline

## Initialize the Clarifai gRPC-based client
Let's import the gRPC-based objects needed to communicate with the Clarifai platform.

In [3]:
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_pb2, status_code_pb2

# Construct the communications channel 
channel = ClarifaiChannel.get_grpc_channel()
# Construct the V2Stub object for accessing all the Clarifai API functionality
stub = service_pb2_grpc.V2Stub(channel)


## Set up Authorization

Set up the metadata object that’s used to authenticate your access to the Clarifai platform.

To create or find a PAT, in Clarifai Community, click on the circular icon for your username in the top right, select “Security”, and then create a PAT or copy an existing one if you’ve already created one. You can follow the below screenshots to see where to find and create PATs.

You can find more information on [this page](https://docs.clarifai.com/clarifai-basics/authentication/authorize/).

<img src="images/notebook1-sk.jpg" alt="drawing" width="800" height="750"/>
<img src="images/notebook2-sk.jpg" alt="drawing" width="800" height="750"/>


<!-- ![](images/notebook1-sk.jpg) -->
<!-- ![](images/notebook2-sk.jpg) -->

In [4]:
PAT = 'YOUR_PAT_HERE'

In [5]:
USER_ID = 'clarifai'
APP_ID = 'main'

# Change these to whatever model and image URL you want to use
MODEL_ID = 'general-image-recognition'
MODEL_VERSION_ID = 'aa7f35c01e0642fda5cf400f543e7c40'
IMAGE_URL = 'https://samples.clarifai.com/metro-north.jpg'

metadata = (('authorization', 'Key ' + PAT),)
userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

## Let's preview the image you used in the URL to see what we're working with

In [6]:
IPython.display.Image(url=IMAGE_URL)

## Collect Your Inputs

Let's grab the images we'll use for making the predictions. 

In [7]:
post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url=IMAGE_URL
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

# Since we have one input, one output will exist here
output = post_model_outputs_response.outputs[0]

print("Predicted concepts:")
for concept in output.data.concepts:
    print("%s %.2f" % (concept.name, concept.value))

Predicted concepts:
train 1.00
railway 1.00
subway system 1.00
station 1.00
locomotive 1.00
transportation system 1.00
travel 0.99
commuter 0.98
platform 0.98
light 0.97
train station 0.97
blur 0.97
city 0.96
road 0.96
urban 0.96
traffic 0.96
street 0.95
public 0.93
tramway 0.93
business 0.93


## Choose a minimum threshold and max concepts

Let's set both a minimum confidence level (ie. we want the model to be at least 80% confidence that the concept is in the image) and a maximum number of concepts to return. We don't want to create prompts with 50 concepts in them!

In [8]:
THRESHOLD = 0.98
MAX_CONCEPTS = 5

## Show the list of concepts we'll use
Here we'll just apply the threshold and maximum so that we can get a nicely formatted list. Since the concepts are return in order, once we hit either limit we can leave the loop


In [9]:
concept_list = []
for count, concept in enumerate(output.data.concepts):
    # Once we have enough concepts, or they're no longer confident enough, quit the loop.
    if (count >= MAX_CONCEPTS) or (round(concept.value, 2) < THRESHOLD):
        break
    concept_list.append(concept.name)    

# Assemble the string by joining the list together separated by commas and terminated with a period.
prompt_end = ", ".join(concept_list) + "."
print(prompt_end)


train, railway, subway system, station, locomotive.


## Let's make some prompts for ChatGPT!
We're going to generate some prompts, print and print them out. For simplicity in this demo we'll copy and paate them into ChatGPT, however with the recent release of the API for ChatGPT this could be automated!

# Travel Agent

In [10]:
travel_prompt = "You are a travel agent. You want to recommend I visit a place \
that contains the following things: "
print(travel_prompt + prompt_end)

You are a travel agent. You want to recommend I visit a place that contains the following things: train, railway, subway system, station, locomotive.


### ChatGPT Response
> Based on your interest in trains, railways, subway systems, stations, and locomotives, I would recommend visiting Tokyo, Japan.

> Tokyo has an extensive railway system, including the famous Shinkansen bullet train that can take you to other parts of Japan at high speeds. The city also has a modern and efficient subway system, which is a popular mode of transportation for locals and tourists alike.

> There are several historic train stations in Tokyo, such as Tokyo Station and Shinjuku Station, which are worth visiting for their impressive architecture and bustling atmosphere. Additionally, there are several train museums and locomotive-themed attractions in the city, such as the Railway Museum and the Omiya Railway Museum.

> Overall, Tokyo is a great destination for train enthusiasts and those interested in transportation history.

# Personal Trainer

In [11]:
trainer_prompt = "You are a personal trainer. Suggest some way I could get in shape using the following things: "
print(trainer_prompt + prompt_end)

You are a personal trainer. Suggest some way I could get in shape using the following things: train, railway, subway system, station, locomotive.


### ChatGPT Response
> While trains, railways, subway systems, stations, and locomotives are not typically associated with fitness, there are still some ways that you can use these elements to help you get in shape:

> 1. Use public transportation: Instead of driving or taking a taxi, consider using public transportation such as trains or subways to get to your destination. Walking to and from the station can add some extra steps to your day and help you burn some extra calories.
2. Stair workouts: Many train stations have multiple levels and lots of stairs. You can use these stairs to do stair workouts, which are great for building lower body strength and cardiovascular endurance. You can also use the stairs as a form of cardio by doing stair sprints or stair jumps.
3. Train station workouts: While waiting for your train, use the benches and walls of the station to do exercises such as push-ups, squats, lunges, and tricep dips. You can also use resistance bands to do a variety of exercises.
4. Train or subway workouts: If you're on a long train or subway ride, use the time to do some bodyweight exercises or stretches. You can do things like squats, lunges, calf raises, or even yoga poses.

> Overall, while trains, railways, subway systems, stations, and locomotives are not traditional fitness tools, there are still some creative ways that you can incorporate them into your workout routine.

# Social Media Influencer

In [12]:
annoying_prompt = "You are a social media influencer. Make a post hyping up all the things in this list, \
how amazing your life is, and how everyone else can be like you if they get these things: "
print(annoying_prompt + prompt_end)

You are a social media influencer. Make a post hyping up all the things in this list, how amazing your life is, and how everyone else can be like you if they get these things: train, railway, subway system, station, locomotive.


### ChatGPT Response
> Hey everyone, are you ready to take your life to the next level? If you want to experience the ultimate in travel and transportation, then you need to get on board with trains, railways, subway systems, stations, and locomotives!

> My life is absolutely amazing thanks to these incredible modes of transportation. From the speedy bullet trains to the classic steam locomotives, I've experienced some of the most breathtaking and unforgettable journeys of my life.

> There's nothing quite like the feeling of riding the rails, gliding past stunning scenery and feeling the wind in your hair. And with so many different options to choose from, you can always find the perfect way to get where you need to go.

> Whether you're commuting to work, traveling across the country, or just exploring your local area, trains, railways, subway systems, stations, and locomotives have everything you need to live your best life. So why wait? Get on board today and start experiencing the incredible adventure that awaits you!

# Financial Analyst

In [13]:
finance_prompt = "You are a financial analyst. Talk about how using the things in this list can benefit \
a person financially: "
print(finance_prompt + prompt_end)

You are a financial analyst. Talk about how using the things in this list can benefit a person financially: train, railway, subway system, station, locomotive.


### ChatGPT Response
> While trains, railways, subway systems, stations, and locomotives may not seem like traditional financial tools, there are actually several ways that they can benefit a person financially:

> 1. Cost-effective transportation: In many cases, using trains, railways, and subways can be a cost-effective alternative to driving or flying. Depending on your location and destination, taking a train or subway may be significantly cheaper than renting a car or buying a plane ticket.
2. Reduced fuel and maintenance costs: If you own a car, using public transportation instead can help you save money on fuel costs and reduce wear and tear on your vehicle. This can lead to lower maintenance costs and a longer lifespan for your car.
3. Time-saving: While taking a train or subway may not always be faster than driving, it can save you time in other ways. For example, you can use the time on the train to work, read, or relax, rather than being stuck in traffic.
4. Reduced stress: Driving in heavy traffic or navigating unfamiliar roads can be stressful and exhausting. By using trains or subways, you can avoid this stress and arrive at your destination feeling refreshed and relaxed.

> Overall, while trains, railways, subway systems, stations, and locomotives may not seem like obvious financial tools, they can offer several financial benefits, including cost-effective transportation, reduced fuel and maintenance costs, time-saving, and reduced stress.

# Wacky Inventor

In [14]:
inventor_prompt = "You are a quirky, eccentric inventor who sees everything in the world as a potential \
invention or gadget. You might come up with wacky and off-the-wall ideas for how to use the things in the \
following list, or you might have a unique perspective on how to approach everyday problems. \
You could add a humorous twist and make your answer more entertaining and engaging. \
Here is the list: "
print(inventor_prompt + prompt_end)

You are a quirky, eccentric inventor who sees everything in the world as a potential invention or gadget. You might come up with wacky and off-the-wall ideas for how to use the things in the following list, or you might have a unique perspective on how to approach everyday problems. You could add a humorous twist and make your answer more entertaining and engaging. Here is the list: train, railway, subway system, station, locomotive.


### ChatGPT Response
> Ah, the wonderful world of transportation! As an inventor, I'm always looking for new and exciting ways to use everyday objects. And trains, railways, subway systems, stations, and locomotives are no exception. Here are a few of my wacky and off-the-wall ideas:

> 1. The Train-ucopia: Why settle for a boring old train ride when you can have an immersive, multi-sensory experience? My Train-ucopia invention would transform the inside of a train car into a magical wonderland of sights, sounds, and smells. Imagine riding through a field of flowers, surrounded by the scent of lavender, or traveling through a rainforest, with the sounds of exotic birds chirping in the background. The Train-ucopia would be the ultimate sensory experience!
2. The Loco-mocha: Who says a locomotive can't also make a mean cup of coffee? My Loco-mocha invention would transform the front of a locomotive into a fully functioning coffee shop. Imagine sipping a latte while you watch the scenery whiz by. Plus, the motion of the train would create the perfect foam for your cappuccino.
3. The Subway Slide: Tired of boring old escalators and stairs in subway stations? My Subway Slide invention would transform the commute into a fun-filled adventure. Instead of taking the stairs, you could slide down a brightly colored slide to get to your platform. It would be the perfect way to start your day with a smile!
4. The Station-scape: Why settle for a drab and dreary train station when you could have a beautiful oasis of greenery? My Station-scape invention would transform train stations into lush gardens filled with flowers, trees, and even a pond. It would be the perfect place to relax and recharge before your train ride.

> Overall, with a little creativity and ingenuity, trains, railways, subway systems, stations, and locomotives can be transformed into all sorts of wacky and wonderful inventions. Who knows what else I'll come up with next?

# Foodie

In [15]:
foodie_prompt = "You are a foodie who sees everything through the lens of culinary creativity. \
You might come up with bizarre recipes or cooking techniques using the items in the list, \
or you might have a unique perspective on how to approach everyday meals. You could add a humorous \
twist to prompts and make them more entertaining and engaging, particularly for those who love \
food and cooking."

print(foodie_prompt + prompt_end)

You are a foodie who sees everything through the lens of culinary creativity. You might come up with bizarre recipes or cooking techniques using the items in the list, or you might have a unique perspective on how to approach everyday meals. You could add a humorous twist to prompts and make them more entertaining and engaging, particularly for those who love food and cooking.train, railway, subway system, station, locomotive.


### ChatGPT Response
Oh, the delicious possibilities of trains, railways, subway systems, stations, and locomotives! As a foodie, I'm always looking for unique ways to incorporate food into every aspect of my life, and transportation is no exception. Here are a few of my bizarre and offbeat ideas:

> 1. Train-wiches: Who needs a boring old sandwich when you can have a train-wich? Imagine a delicious sub sandwich, served inside a miniature train car. You could choose from a variety of flavors, like the "Choo-Choo Chicken" or the "Loco-Meatball."
2. Subway Snacks: Why settle for a plain old bag of chips when you can have subway snacks? These bite-sized treats would be served in miniature subway cars, each one filled with a different flavor. You could choose from savory options like "Meatball Marinara" or "Veggie Delight," or indulge your sweet tooth with flavors like "Cookies and Cream" or "Chocolate Fudge."
3. Station Stew: Need a hearty meal to warm you up on a chilly day? Look no further than Station Stew! This flavorful dish would be served in miniature train station buildings, with a variety of toppings like cheese, croutons, and bacon bits. It's the perfect way to fuel up before a long train ride.
4. Locomotive Lattes: Need a pick-me-up before your morning commute? Look no further than Locomotive Lattes! These delicious coffee drinks would be served inside miniature locomotives, with a variety of flavors to choose from. And of course, they'd be topped with a frothy layer of foam shaped like a train engine.

Overall, with a little culinary creativity, trains, railways, subway systems, stations, and locomotives can be transformed into delicious and delightful culinary experiences. All aboard the foodie express!

# Rapper

In [16]:
rapper_prompt = "You are a rapper obsessed with the items listed at the end of this prompt. Write a \
fun, offbeat rap that praises everything in the list, and how you are so grateful that you can \
afford everything in the list now that you are a successful rapper. Here is the list: "

print(rapper_prompt + prompt_end)

You are a rapper obsessed with the items listed at the end of this prompt. Write a fun, offbeat rap that praises everything in the list, and how you are so grateful that you can afford everything in the list now that you are a successful rapper. Here is the list: train, railway, subway system, station, locomotive.


### ChatGPT Response
> Verse 1:
I used to walk around the city on my feet
But now I ride the train, feelin' so elite
Got the railway on lock, yeah, I'm the king
Rollin' through the subway, hear the people sing

> Chorus:
Train, railway, subway, station
Locomotive, man, that's my motivation
I used to walk, now I ride in style
Thanks to my success, I can afford this lifestyle

> Verse 2:
I used to wait in traffic, wasting all my time
Now I take the train, everything's just fine
I roll through the station, people waving at me
I'm living the dream, can't you see?

> Chorus:
Train, railway, subway, station
Locomotive, man, that's my motivation
I used to walk, now I ride in style
Thanks to my success, I can afford this lifestyle

> Verse 3:
I used to dream of a life like this
Ridin' in the train, feeling so bliss
But now I'm living it, everything's so sweet
I roll through the city, everyone wants to meet

> Chorus:
Train, railway, subway, station
Locomotive, man, that's my motivation
I used to walk, now I ride in style
Thanks to my success, I can afford this lifestyle

> Outro:
So if you want to ride like me
Just work hard, you'll see
The train, railway, subway, station
Locomotive, man, that's the key to elevation!